<a href="https://colab.research.google.com/github/Hamza-Suleman/transiting-exoplanets/blob/main/Wrapper_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>